Scrape reddit focusing on r/electricvehicles, r/EVcharging, and r/Tesla and look for certain keywords.

Clean and preprocess the text data (remove stop words, tokenize).

Perform topic modeling (e.g., LDA) to identify key themes/problems.

Use pre-trained sentiment models like VADER or fine-tune your own.

Classify sentiment as positive, negative, or neutral to gauge user satisfaction.

EV Scraper
personal use script
EAEzPWfS3NsqvM3mgI_9KQ


secret	cmtE_-zKxIBzoVPW8nf4jEMefYpkOQ

In [11]:
%pip install praw
%pip install requests
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 40.1 MB/s eta 0:00:00


In [13]:
import praw
import requests
import pandas as pd
from datetime import datetime

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
reddit = praw.Reddit(
    client_id = 'EAEzPWfS3NsqvM3mgI_9KQ',
    client_secret = 'cmtE_-zKxIBzoVPW8nf4jEMefYpkOQ',
    user_agent="hyltda",
)

print(reddit.read_only)

True


In [5]:
def scrape_reddit(subreddits, keywords, limit=100):
    data = []

    for subreddit_name in subreddits:
        subreddit = reddit.subreddit(subreddit_name)
        for post in subreddit.search(" OR ".join(keywords), limit=limit):
            post_data = {
                "title": post.title,
                "body": post.selftext,
                "upvotes": post.score,
                "comments": [],
                "url": post.url,
            }

            post.comments.replace_more(limit=None)
            for comment in post.comments.list():
                post_data["comments"].append(comment.body)
            data.append(post_data)

    return pd.DataFrame(data)


subreddits = ["electricvehicles", "EVcharging", "Tesla"]
keywords = ["charging issues", "range anxiety", "maintenance problems"]

df = scrape_reddit(subreddits, keywords, limit=50)
print(df.head())

                                               title  \
0  Why I, a prospective EV buyer, am not buying a...   
1      2014 Ford Focus Electric - 3 months owning it   
2  Cable problems crop up as non-Tesla EVs begin ...   
3  Electrify America Explains Cause Of Those Vexi...   
4  The Southern California EV Charging Problem! (...   

                                                body  upvotes  \
0  There’s been a lot of posts, articles, and ana...        0   
1  Hello everyone,\n\nI understand there is a gre...       37   
2                                                         266   
3  https://www.forbes.com/sites/brookecrothers/20...      231   
4                                                         100   

                                            comments  \
0  [[deleted], Why I, a prospective eater of all ...   
1  [Good plan! Thank you for sharing your experie...   
2  [2023 - the media discover... Charge port loca...   
3  [“A lot of people are frustrated by broken ch

In [6]:
df.to_csv("reddit_ev_issues.csv", index=False)

In [7]:
df = pd.read_csv('reddit_ev_issues.csv')

print(df.head())
print(df.info())

                                               title  \
0  Why I, a prospective EV buyer, am not buying a...   
1      2014 Ford Focus Electric - 3 months owning it   
2  Cable problems crop up as non-Tesla EVs begin ...   
3  Electrify America Explains Cause Of Those Vexi...   
4  The Southern California EV Charging Problem! (...   

                                                body  upvotes  \
0  There’s been a lot of posts, articles, and ana...        0   
1  Hello everyone,\n\nI understand there is a gre...       37   
2                                                NaN      266   
3  https://www.forbes.com/sites/brookecrothers/20...      231   
4                                                NaN      100   

                                            comments  \
0  ['[deleted]', 'Why I, a prospective eater of a...   
1  ['Good plan! Thank you for sharing your experi...   
2  ['2023 - the media discover... Charge port loc...   
3  ['“A lot of people are frustrated by broken c

### Preprocessing the text

In [15]:
df.columns

Index(['title', 'body', 'upvotes', 'comments', 'url'], dtype='object')

In [19]:
# Download necessary resources
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

def preprocess_text(text):
    # Remove URLs and special characters
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"[^\w\s]", "", text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize and remove stopwords
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words("english")]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return " ".join(tokens)

# Apply preprocessing to the "title" column
df["clean_text"] = df["title"].fillna("").apply(preprocess_text)

# Preview cleaned text and the original title
print(df[["title", "clean_text"]].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ysuns\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ysuns\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ysuns\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Ysuns/nltk_data'
    - 'c:\\Desktop\\coding\\MYDAC-Curriculum\\.venv\\nltk_data'
    - 'c:\\Desktop\\coding\\MYDAC-Curriculum\\.venv\\share\\nltk_data'
    - 'c:\\Desktop\\coding\\MYDAC-Curriculum\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\Ysuns\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
